# 1. Knowledge Graphs

---

### Setup & Imports

---

### A toy knowledge graph

---

### LLMs can extract knowledge graphs from text!

---

### Querying the knowledge graph

---

### Viewing the knowledge graph with Neo4j (is awesome)